In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import mysql.connector
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv("C:/Users/lenovo/Downloads/online_book_dataset/books.csv")

In [ ]:
# Data Cleaning :Convert rating text to numeric
rating_map = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
df['rating'] = df['rating'].map(rating_map)

# Add binary column for stock availability
df['in_stock'] = df['stock'].apply(lambda x: 1 if 'In stock' in x else 0)

In [ ]:
# Drop unnecessary column
df_cleaned = df.drop(columns=['book_url'])

In [ ]:
# Replace NaN with None to handle SQL NULL
df_cleaned =df_cleaned.where(pd.notnull(df_cleaned),None)
    
# Debugging: Check for NaN values
print(df_cleaned.isnull().sum())

# Exploratory Data Analysis (EDA)

In [ ]:
# Total number of books in dataset
print(df_cleaned.shape[0])

In [ ]:
# Total number of unique book categories
print(df_cleaned['category'].nunique())


In [ ]:
# Average price of all books
print(df_cleaned['price'].mean())

In [ ]:
# Count of books with a perfect 5-star rating
print((df_cleaned['rating'] == 5).sum())

In [ ]:
# 10 most expensive books
df_cleaned.sort_values(by='price', ascending=False).head(10)[['title', 'category', 'price']]


In [ ]:
#Categories with Highest Median Price
df_cleaned.groupby('category')['price'].median().sort_values(ascending=False).head(10)

In [ ]:
# Distribution of Book Ratings per Category
df_cleaned.groupby(['category', 'rating']).size().unstack(fill_value=0).head(10)

In [ ]:
#Categories with Only High Ratings (4 or 5)
high_rating_only = df_cleaned.groupby('category')['rating'].min()
high_rating_only[high_rating_only >= 4]

In [ ]:
#Price Range Summary (Mean, Median, Min, Max, Std)
df_cleaned['price'].agg(['mean', 'median', 'min', 'max', 'std'])

In [ ]:
#Correlation Between Price and Rating
df_cleaned[['price', 'rating']].corr().iloc[0, 1]


In [ ]:
#Rating Buckets (Low: 1–2, Mid: 3, High: 4–5)
df_cleaned['rating_bucket'] = pd.cut(df['rating'], bins=[0, 2, 3, 5], labels=['Low', 'Mid', 'High'])
df_cleaned['rating_bucket'].value_counts()

In [ ]:
df_cleaned['price_bucket'] = pd.cut(
    df_cleaned['price'],
    bins=[0, 20, 40, 60, float('inf')],
    labels=['Below 20', '20-40', '41-60', 'Above 60']
)


In [ ]:
df_cleaned['price_bucket'].value_counts().sort_index()


In [ ]:
df_cleaned['rating_bucket'] = pd.cut(df_cleaned['rating'], bins=[0, 2, 3, 5], labels=['Low', 'Mid', 'High'])


In [ ]:
# Now create the crosstab
pd.crosstab(df_cleaned['price_bucket'], df_cleaned['rating_bucket'])

# Data Visualization

In [ ]:
# Price bucket distribution
avg_price_by_rating = df_cleaned.groupby('rating')['price'].mean().round(2)
top_categories = df_cleaned['category'].value_counts().head(5)

sns.countplot(x='price_bucket', data=df_cleaned, order=['Below 20', '20-40', '41-60', 'Above 60'])
plt.title('Price Range Distribution')
plt.xlabel('Price Range')
plt.ylabel('Number of Books')
plt.show()

In [ ]:
# Average price by rating
sns.barplot(x=avg_price_by_rating.index, y=avg_price_by_rating.values)
plt.title('Average Price by Rating')
plt.xlabel('Rating')
plt.ylabel('Average Price')
plt.show()

In [ ]:
# Top 5 book categories
top_categories.plot(kind='bar', color='skyblue')
plt.title('Top 5 Book Categories')
plt.ylabel('Number of Books')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Rating Distribution – Horizontal with color

rating_dist = df_cleaned['rating'].value_counts().sort_index().reset_index()
rating_dist.columns = ['rating', 'count']

plt.figure(figsize=(7, 4))
sns.barplot(data=rating_dist, x='count', y='rating', palette='Blues_d')
plt.title('Book Rating Distribution')
plt.xlabel('Number of Books')
plt.ylabel('Rating')
plt.tight_layout()
plt.show()

In [ ]:
# Price Distribution – Histogram with KDE
plt.figure(figsize=(8, 4))
sns.histplot(df_cleaned['price'], kde=True, color='teal', bins=30)
plt.title('Price Distribution of Books')
plt.xlabel('Price')
plt.tight_layout()
plt.show()

In [ ]:
# Group by rating and count
df_rating = df_cleaned['rating'].value_counts().sort_index(ascending=False).reset_index()
df_rating.columns = ['rating', 'total']

# Step 2: Plot
plt.figure(figsize=(6, 4))
sns.barplot(data=df_rating, x='rating', y='total', palette='Blues')
plt.title('Distribution of Book Ratings')
plt.tight_layout()
plt.show()

# Connecting to MySQL

In [ ]:
# Function to map pandas dtype to SQL
def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'DATETIME'
    else:
        return 'TEXT'

In [ ]:
# Connect to MySQL
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Miock#0759',
    database='books_analysis'
)
cursor = conn.cursor()

In [ ]:
# Create table
table_name = 'online_books'

# Generate column definitions based on DataFrame
columns = ', '.join([
    f'`{col}` {get_sql_type(df_cleaned[col].dtype)}'
    for col in df_cleaned
])

# Build and execute the CREATE TABLE SQL query
create_query = f'''
CREATE TABLE IF NOT EXISTS `{table_name}` (
    {columns}
)
'''
cursor.execute(create_query)

In [ ]:
for _, row in df_cleaned.iterrows():
    values = tuple(row)
    placeholders = ', '.join(['%s'] * len(row))
    insert_query = f'INSERT INTO `{table_name}` ({", ".join("`" + col + "`" for col in df_cleaned.columns)}) VALUES ({placeholders})'

    cursor.execute(insert_query, values)

In [ ]:
# Commit the transaction for the current CSV file
conn.commit()

In [ ]:
# Close the connection
conn.close()
print("Cleaned data inserted into MySQL.")

In [ ]:
db=mysql.connector.connect(host="localhost", username="root", password="Miock#0759", database="books_analysis")
cur=db.cursor()

# MySQL QUERIES

In [ ]:
#1. List of all unique categories
query ="""SELECT DISTINCT category FROM online_books"""
cur.execute(query)
data= cur.fetchall()
data

In [ ]:
#2. Top 5 most common categories
query ="""SELECT category, COUNT(*) AS total
FROM online_books
GROUP BY category
ORDER BY total DESC
LIMIT 5"""
cur.execute(query)
data= cur.fetchall()
data

In [ ]:
#3. Average price by rating
query ="""SELECT rating, ROUND(AVG(price), 2) AS avg_price
FROM online_books
GROUP BY rating
ORDER BY rating"""
cur.execute(query)
data= cur.fetchall()
data

In [ ]:
#4. Number of in-stock vs out-of-stock books
query ="""SELECT stock, COUNT(*) AS total
FROM online_books
GROUP BY stock"""
cur.execute(query)
data= cur.fetchall()
data


In [ ]:
#5. Top 3 most expensive books per category
query ="""SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY category ORDER BY price DESC) AS rnk
    FROM online_books
) ranked
WHERE rnk <= 3"""
cur.execute(query)
data= cur.fetchall()
data


In [ ]:
#6. Category with highest average price
query ="""SELECT category, ROUND(AVG(price), 2) AS avg_price
FROM online_books
GROUP BY category
ORDER BY avg_price DESC
LIMIT 1"""
cur.execute(query)
data= cur.fetchall()
data


In [ ]:
#7. Category with most 5-star rated books
query ="""SELECT category, COUNT(*) AS five_star_books
FROM online_books
WHERE rating = 5
GROUP BY category
ORDER BY five_star_books DESC
LIMIT 1"""
cur.execute(query)
data= cur.fetchall()
data

In [ ]:
#8. Price distribution buckets
query ="""SELECT
  CASE
    WHEN price < 20 THEN 'Below 20'
    WHEN price BETWEEN 20 AND 40 THEN '20-40'
    WHEN price BETWEEN 41 AND 60 THEN '41-60'
    ELSE 'Above 60'
  END AS price_range,
  COUNT(*) AS total_books
FROM online_books
GROUP BY price_range
ORDER BY price_range"""
cur.execute(query)
data= cur.fetchall()
data


In [ ]:
#9. Category with widest price range (Max - Min)
query ="""SELECT category, MAX(price) - MIN(price) AS price_range
FROM online_books
GROUP BY category
ORDER BY price_range DESC
LIMIT 1"""
cur.execute(query)
data= cur.fetchall()
data



In [ ]:
#10. Category with highest price volatility (standard deviation)
query ="""SELECT category, ROUND(STDDEV(price), 2) AS price_stddev
FROM online_books
GROUP BY category
ORDER BY price_stddev DESC
LIMIT 1"""
cur.execute(query)
data= cur.fetchall()
data



In [ ]:
#11. Percentage of books rated 4 stars or above
query ="""SELECT 
  ROUND(SUM(CASE WHEN rating >= 4 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS high_rating_percentage
FROM online_books"""
cur.execute(query)
data= cur.fetchall()
data



In [ ]:
#12. Average price: In-stock vs Out-of-stock
query ="""SELECT stock, ROUND(AVG(price), 2) AS avg_price
FROM online_books
GROUP BY stock"""
cur.execute(query)
data= cur.fetchall()
data

In [ ]:
#13.Categories where all books are priced above the overall average
query =""" SELECT category
FROM online_books
WHERE category NOT IN (
    SELECT category
    FROM online_books
    WHERE price <= (
        SELECT AVG(price) FROM online_books
    )
)
GROUP BY category """
cur.execute(query)
data= cur.fetchall()
data



In [ ]:
#14. Top 5 categories with the most ‘High’ rated books (4–5)
query ="""SELECT category, COUNT(*) AS high_rated_books
FROM online_books
WHERE rating IN (4, 5)
GROUP BY category
ORDER BY high_rated_books DESC
LIMIT 5"""
cur.execute(query)
data= cur.fetchall()
data


In [ ]:
#15. Average price per star rating per stock status
query ="""SELECT rating, stock, ROUND(AVG(price), 2) AS avg_price
FROM online_books
GROUP BY rating, stock
ORDER BY rating, stock"""
cur.execute(query)
data= cur.fetchall()
data


In [ ]:
#16. List books with price above category average
query ="""SELECT b.*
FROM online_books b
JOIN (
  SELECT category, AVG(price) AS avg_cat_price
  FROM online_books
  GROUP BY category
) c_avg
ON b.category = c_avg.category
WHERE b.price > c_avg.avg_cat_price
LIMIT 10"""
cur.execute(query)
data= cur.fetchall()
data


In [ ]:
#17. Category-rating pairs with less than 3 books
query ="""SELECT category, rating, COUNT(*) AS total
FROM online_books
GROUP BY category, rating
HAVING COUNT(*) < 3 
LIMIT 5"""
cur.execute(query)
data= cur.fetchall()
data



In [ ]:
#18. Top-rated book (price-wise) per category
query ="""SELECT *
FROM online_books b
WHERE price = (
  SELECT MAX(price)
  FROM online_books
  WHERE category = b.category
)
ORDER BY category
LIMIT 5"""
cur.execute(query)
data= cur.fetchall()
data



